In [1]:
#!groovy

In [2]:

// Download dependencies using Groovy's Grape system
import groovy.grape.Grape
Grape.grab(group:'org.semanticweb.elk', module:'elk-owlapi', version:'0.4.3')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-api', version:'4.2.5')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-apibinding', version:'4.2.5')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-impl', version:'4.2.5')
Grape.grab(group:'com.github.sharispe', module:'slib-sml', version:'0.9.1')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-parsers', version:'4.2.5')

# Class metadata parser

This notebook is intended to parse ID, name and synonyms for each class in the owl ontology input.

## DOID

In [15]:
// import what we need
import java.net.*
import org.semanticweb.owlapi.model.parameters.*
import org.semanticweb.elk.owlapi.ElkReasonerFactory;
import org.semanticweb.elk.owlapi.ElkReasonerConfiguration
import org.semanticweb.elk.reasoner.config.*
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.reasoner.*
import org.semanticweb.owlapi.reasoner.structural.StructuralReasoner
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.*;
import org.semanticweb.owlapi.owllink.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.search.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.reasoner.structural.*


// define variables
Map<String, Set<String>> id2name = [:].withDefault { new TreeSet() }
Map<String, Set<String>> id2syn = [:].withDefault { new TreeSet() }

// create parser function
def id = ""
OWLOntologyManager manager = OWLManager.createOWLOntologyManager()  
OWLDataFactory fac = manager.getOWLDataFactory()

 def parseOntologies = { ontUriOrfile -> 

  ontologyfile = manager.loadOntologyFromOntologyDocument(ontUriOrfile)

  ontologyfile.getClassesInSignature(true).each {
   cl ->
    class_ = cl.toString()
    id = class_.substring(class_.lastIndexOf('/')+1,class_.length()-1).replaceAll('_',':')
    EntitySearcher.getAnnotationObjects(cl, ontologyfile, fac.getRDFSLabel()).each {
     lab ->
      if (lab.getValue() instanceof OWLLiteral) {
       def labs = (OWLLiteral) lab.getValue()
       name = labs.getLiteral().trim().toLowerCase()
       id2name[id].add(name)
      }
    }

    EntitySearcher.getAnnotationAssertionAxioms(cl, ontologyfile).each {
      ax ->
       if (ax.getProperty().toString() == "<http://www.geneontology.org/formats/oboInOwl#hasDbXref>") {
          if (ax.getValue()!= null && ax.getValue()!= [] && !(ax.getValue() instanceof IRI)){
            def syn = ax.getValue()
            if(!(ax.getValue() instanceof String) ){
              syn = ax.getValue().getLiteral()
            }
            id2syn[id].add(syn)    
          }
          
       }else if(ax.getProperty().toString() == "<http://www.geneontology.org/formats/oboInOwl#hasExactSynonym>"){
          if (ax.getValue()!= null && ax.getValue()!= [] && !(ax.getValue() instanceof IRI)){
            def syn = ax.getValue()
            if(!(ax.getValue() instanceof String) ){
              syn = ax.getValue().getLiteral()
            }
            id2syn[id].add(syn)
          }
       }
    }
  }
     
  println "finished parsing\n"
 }
     
// Parse ontologies
parseOntologies(IRI.create("http://purl.obolibrary.org/obo/doid.owl"))
    
// write out metadata     
println id2name.get(1)

[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] INFO org.semanticweb.owlapi.io.AbstractOWLParser - URL connection input stream is compressed using gzip
[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] INFO org.semanticweb.owlapi.io.AbstractOWLParser - URL connection input stream is compressed using gzip
[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] INFO org.semanticweb.owlapi.io.AbstractOWLParser - URL connection input stream is compressed using gzip
[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] INFO org.semanticweb.owlapi.io.AbstractOWLParser - URL connection input stream is compressed using gzip
[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] INFO org.semanticweb.owlapi.io.AbstractOWLParser - URL connection input stream is compressed using gzip
[SciJava-29073b67-Thread-org.scijava.jupyter.kernel.evaluator.ScijavaEvaluator] 

finished parsing

[]
